In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.widgets import Cursor
import PySimpleGUI as sg
from sklearn.linear_model import LinearRegression 
import warnings

# Suppress the specific UserWarning from Matplotlib's tight_layout
warnings.filterwarnings("ignore", message="The figure layout has changed to tight", category=UserWarning)

# --- Core Pressure Calculation Functions ---
def calculate_pressure_at_depth(depth, known_pressure, known_depth, pressure_gradient):
    """
    Calculates pressure at a given depth based on a known pressure at a known depth
    and a constant pressure gradient.

    Args:
        depth (float or np.array): The depth(s) at which to calculate pressure (ft).
        known_pressure (float): Pressure at the known depth (psia).
        known_depth (float): The known reference depth (ft).
        pressure_gradient (float): The pressure gradient (psi/ft).

    Returns:
        float or np.array: Pressure at the given depth(s) in psia.
    """
    return known_pressure + pressure_gradient * (depth - known_depth)

# --- GUI Layout ---
def create_gui_layout():
    """Defines the PySimpleGUI layout for user inputs and includes the plot canvas."""
    input_column = [
        [sg.Text("Reservoir Pressure Profile & OWC Calculator", font=("Helvetica", 16, "bold"), justification='center', expand_x=True)],
        [sg.HSeparator()],
        [sg.Text("Pressure Gradients:", font=("Helvetica", 12, "bold"))],
        [sg.Text("Oil Pressure Gradient (Pg_o):", size=(30, 1)), sg.InputText("0.35", key='-PG_O-'), sg.Text("psi/ft")],
        [sg.Text("Gas Pressure Gradient (Pg_g):", size=(30, 1)), sg.InputText("0.08", key='-PG_G-'), sg.Text("psi/ft")],
        [sg.Text("Normal Pressure Gradient (Pg_n):", size=(30, 1)), sg.InputText("0.45", key='-PG_N-'), sg.Text("psi/ft")],
        [sg.HSeparator()],
        [sg.Text("Depths & Well Test Data:", font=("Helvetica", 12, "bold"))],
        [sg.Text("Reservoir Top Depth (res_top):", size=(30, 1)), sg.InputText("5000", key='-RES_TOP-'), sg.Text("ft")],
        [sg.Text("Gas-Oil Contact (GOC) Depth:", size=(30, 1)), sg.InputText("5200", key='-GOC-'), sg.Text("ft")],
        [sg.Text("Well Test Pressure (Ptest):", size=(30, 1)), sg.InputText("2402", key='-P_TEST-'), sg.Text("psia")],
        [sg.Text("Well Test Depth (Dtest):", size=(30, 1)), sg.InputText("5250", key='-D_TEST-'), sg.Text("ft")],
        [sg.HSeparator()],
        [sg.Push(), sg.Button("Generate Plot", key='-GENERATE_PLOT-', size=(15, 2)), sg.Button("Exit", size=(10, 2)), sg.Push()]
    ]

    # Matplotlib Canvas where the plot will be embedded
    plot_column = [
        [sg.Canvas(key='-CANVAS-', size=(600, 800))] # Adjust size as needed
    ]

    layout = [
        [sg.Column(input_column, vertical_alignment='top'),
         sg.VSeperator(),
         sg.Column(plot_column, vertical_alignment='top')]
    ]
    return sg.Window("Reservoir Pressure Profile & OWC", layout, finalize=True, element_justification='left')

# --- Main Application Logic ---
def main():
    window = create_gui_layout()
    canvas_elem = window['-CANVAS-'] # Get the canvas element once

    # --- Initialize Matplotlib Figure and Embed Once ---
    fig, ax = plt.subplots(figsize=(6, 8), dpi=100) # Create figure and axes
    fig.patch.set_facecolor(sg.theme_background_color()) # Match theme if needed

    # Embed the Matplotlib figure into the PySimpleGUI canvas
    figure_canvas_agg = FigureCanvasTkAgg(fig, canvas_elem.TKCanvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1)

    # --- Container for dynamic plot elements ---
    # This dictionary will hold references to the plot lines, annotation, and cursor.
    # The hover_handler will always access these through this dictionary.
    plot_elements = {
        'line_actual': None,
        'line_normal': None,
        'annot': None,
        'cursor': None
    }

    # --- Define Hover Function Once and Connect Once ---
    def hover_handler(event):
        # Access plot elements through the dictionary
        current_ax = ax # ax is constant, captured from main scope
        current_figure_canvas_agg = figure_canvas_agg # canvas is constant

        # Crucial check: Ensure annotation object exists before trying to use it
        # This prevents AttributeError if mouse moves before initial plot or during clear/redraw cycle
        if plot_elements['annot'] is None:
            return # Exit if annotation hasn't been initialized yet

        if event.inaxes == current_ax:
            hit_line = None
            # Check both lines from the plot_elements dictionary
            for line in [plot_elements['line_actual'], plot_elements['line_normal']]:
                if line: # Ensure line object exists before checking its 'contains' method
                    contains, _ = line.contains(event)
                    if contains:
                        hit_line = line
                        break

            if hit_line:
                x_data, y_data = hit_line.get_data()
                # Find the closest point in the line's data to the mouse's y-coordinate
                closest_index = np.argmin(np.abs(y_data - event.ydata))
                display_x = x_data[closest_index]
                display_y = y_data[closest_index]

                plot_elements['annot'].xy = (display_x, display_y)
                plot_elements['annot'].set_text(f"Depth: {display_y:.2f} ft\nPressure: {display_x:.2f} psia")
                plot_elements['annot'].set_visible(True)
                current_figure_canvas_agg.draw_idle()
                return

        # If mouse not over a line, hide annotation
        # This check is safe now because we already know plot_elements['annot'] is not None
        if plot_elements['annot'].get_visible():
            plot_elements['annot'].set_visible(False)
            current_figure_canvas_agg.draw_idle()

    # Connect the hover event listener to the figure canvas once
    figure_canvas_agg.mpl_connect("motion_notify_event", hover_handler)

    while True:
        event, values = window.read()

        if event == sg.WIN_CLOSED or event == 'Exit':
            break

        if event == '-GENERATE_PLOT-':
            try:
                # --- Get Inputs from GUI ---
                Pg_o = float(values['-PG_O-'])
                Pg_g = float(values['-PG_G-'])
                Pg_n = float(values['-PG_N-'])
                res_top = float(values['-RES_TOP-'])
                GOC = float(values['-GOC-'])
                Ptest = float(values['-P_TEST-'])
                Dtest = float(values['-D_TEST-'])

                # --- Input Validation ---
                if not (Pg_o > Pg_g and Pg_n > 0):
                    sg.popup_error("Input Error: Oil gradient must be greater than gas gradient. Gradients must be positive.")
                    continue
                if not (res_top < GOC < Dtest):
                    sg.popup_error("Input Error: Depths must be in ascending order: Reservoir Top < GOC < Test Depth.")
                    continue
                if Ptest <= 0:
                    sg.popup_error("Input Error: Well Test Pressure must be positive.")
                    continue

                # --- Calculations for Actual Fluid Pressure Profile ---
                PGOC = calculate_pressure_at_depth(GOC, Ptest, Dtest, Pg_o)
                P_top = calculate_pressure_at_depth(res_top, PGOC, GOC, Pg_g)

                min_plot_depth = res_top - 100
                max_plot_depth = Dtest + 1000
                depths_full_range = np.linspace(min_plot_depth, max_plot_depth, 500)

                P_actual_fluid = np.zeros_like(depths_full_range)
                for i, d in enumerate(depths_full_range):
                    if d <= GOC:
                        P_actual_fluid[i] = calculate_pressure_at_depth(d, PGOC, GOC, Pg_g)
                    else:
                        P_actual_fluid[i] = calculate_pressure_at_depth(d, PGOC, GOC, Pg_o)

                # --- Calculations for Normal Pressure Profile ---
                P_normal_at_Dtest = (Pg_n * Dtest) + 14.7
                P_normal_profile = calculate_pressure_at_depth(depths_full_range, P_normal_at_Dtest, Dtest, Pg_n)

                # --- Update Plotting Data: Clear and Re-plot all elements ---
                ax.clear() # Clear existing content from the axes

                # Re-plot the lines and store NEW references in the plot_elements dictionary
                plot_elements['line_actual'], = ax.plot(P_actual_fluid, depths_full_range, label='Actual Fluid Pressure', color='red', linestyle='-')
                plot_elements['line_normal'], = ax.plot(P_normal_profile, depths_full_range, label='Normal Pressure (Brine)', color='blue', linestyle='--')

                # Re-set plot properties (labels, title, limits, etc.) after clearing
                ax.set_xlabel("Pressure (psia)")
                ax.set_ylabel("True Vertical Depth (ft)")
                ax.set_title("Reservoir Pressure Profiles & Oil-Water Contact (OWC)")
                ax.invert_yaxis()
                ax.grid(True, linestyle=':', alpha=0.7)
                
                # Re-add legend with the new line objects from plot_elements
                ax.legend([plot_elements['line_actual'], plot_elements['line_normal']], ['Actual Fluid Pressure', 'Normal Pressure (Brine)'])
                
                ax.xaxis.set_ticks_position('top')
                ax.xaxis.set_label_position('top')
                ax.tick_params(axis='x', rotation=45) # Rotate x-axis labels if they overlap

                # Re-add key points and their texts (they were cleared by ax.clear())
                ax.plot(P_top, res_top, 'ko', markersize=6)
                ax.text(P_top, res_top - 50, f'Res Top: {P_top:.0f} psia', verticalalignment='top', horizontalalignment='right')
                ax.plot(PGOC, GOC, 'go', markersize=6)
                ax.text(PGOC, GOC - 50, f'GOC: {PGOC:.0f} psia', verticalalignment='top', horizontalalignment='left')
                ax.plot(Ptest, Dtest, 'mo', markersize=6)
                ax.text(Ptest, Dtest + 50, f'Test: {Ptest:.0f} psia', verticalalignment='bottom', horizontalalignment='right')

                # Re-initialize annotation and cursor on the new ax context after clear
                # The old cursor will be replaced. We remove the explicit .clear() call.
                plot_elements['annot'] = ax.annotate("", xy=(0,0), xytext=(20,-20),textcoords="offset points",
                                    bbox=dict(boxstyle="round,pad=0.5", fc="yellow", alpha=0.7),
                                    arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0"))
                plot_elements['annot'].set_visible(False)
                plot_elements['cursor'] = Cursor(ax, useblit=True, color='green', linewidth=1) # Create new cursor on the new ax context

                # Auto-scale view to fit new data
                ax.autoscale_view()
                fig.tight_layout() # Adjust layout for new data/limits

                # Draw the updated plot on the embedded canvas
                figure_canvas_agg.draw()

            except ValueError:
                sg.popup_error("Invalid input. Please ensure all fields are numeric.")
            except Exception as e:
                sg.popup_error(f"An unexpected error occurred: {e}. Please check your inputs.")

    # Clean up Matplotlib figure when GUI closes
    if fig:
        plt.close(fig)
    window.close()

if __name__ == "__main__":
    main()